In [1]:
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.utils import upload_to_neuroglancer as upload
from brainlit.utils import upload_skeleton
import numpy as np
from pathlib import Path

In [2]:
import napari
%gui qt

In [5]:
data_dir = str(Path().resolve().parents[2] / "tests" / "data_octree")
dest_dir = str(Path() / "upload")

num_res = 2

# Uploading Brain Images in the Octree Format
## This is a script for uploading entire brain volumes, or uploading specific resolutions onto AWS or a local directory. 
## Data must be tif files arranged in folders where the highest level corresponds to a single, low res image
### Files should be arranged as octree with 1-8 indicating volume octant, Binary paths are used to stitch together images according to resolution

In [4]:
files, bin_paths, vox_size, tiff_dims = upload.get_volume_info(data_dir, num_res, channel = 0)
print("Low res files: " + str(files[0]))
print("\nHigh res files: " + str(files[1]))
print("---")
print("Single image binary: " + str(bin_paths[0]))
print("\nMultiple image binaries: " + str(bin_paths[1]))

got files and binary representations of paths.
got dimensions of volume
Low res files: ['/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/default.0.tif']

High res files: ['/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/7/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/6/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/1/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/8/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/4/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/3/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/2/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/5/default.0.tif']
---
Single image binary: [[]]

Multiple image binaries: [['110'], ['101'], ['

### Cloudvolume image layers are created with the number of resolutions in the original data

In [5]:
vols = upload.create_image_layer("file://" + dest_dir,tiff_dims, vox_size, num_res)
print("Number of volumes: " + str(len(vols)))
print("mips: " + str(vols[0].mip) + ' and ' + str(vols[1].mip))
print("Volumes info: " + str(vols[0].info))
print("---")
print("High res volume info: " + str(vols[0].info['scales'][0]))
print("\nLow res volume info: " + str(vols[1].info['scales'][1]))

Number of volumes: 2
mips: 1 and 0
Volumes info: {'data_type': 'uint16', 'num_channels': 1, 'scales': [{'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '6173_6173_6173', 'resolution': [6173, 6173, 6173], 'size': [1056, 800, 416], 'voxel_offset': [0, 0, 0]}, {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '12346_12346_12346', 'resolution': [12346, 12346, 12346], 'size': [528, 400, 208], 'voxel_offset': [0, 0, 0]}], 'type': 'image'}
---
High res volume info: {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '6173_6173_6173', 'resolution': [6173, 6173, 6173], 'size': [1056, 800, 416], 'voxel_offset': [0, 0, 0]}

Low res volume info: {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '12346_12346_12346', 'resolution': [12346, 12346, 12346], 'size': [528, 400, 208], 'voxel_offset': [0, 0, 0]}


### Uploading can be done with either Joblib parallel or non-parrallel sequential if the cpu power isn't there

In [12]:
%%capture
u1=upload.upload_chunks(vols[0], files[0], bin_paths[0], parallel=False) # Low res
u2=upload.upload_chunks(vols[1], files[1], bin_paths[1], parallel=False) # High res

# Visualize your data with NeuroglancerSession

In [19]:
mip = 0 # this can be either 0 or 1
tiff_dims = [528*(2-mip),400*(2-mip),208*(2-mip)]
ngl_sess = NeuroglancerSession(mip = mip, url = "file://" + dest_dir)
from cloudvolume import Bbox
img = ngl_sess.pull_bounds_img(Bbox((0,0,0), tiff_dims))

Downloading:   0%|          | 0/256 [00:00<?, ?it/s]
Downloading: 424it [00:00, 641.30it/s]
Downloading: 486it [00:00, 712.58it/s]
Downloading: 358it [00:01, 305.47it/s]
Downloading: 448it [00:01, 446.07it/s]
Downloading: 504it [00:01, 266.37it/s]
Downloading: 488it [00:00, 596.86it/s]


In [20]:
with napari.gui_qt():
    ngl_sess.napari_viewer(img)

In [4]:
# We can also load SWC files

In [3]:
# Uploading Neuron traces in .swc format locally
swc_dir = str(Path().resolve().parents[2] / "tests" / "data_swcs")
dest_dir_skel = str(Path() / "upload")

In [6]:
data_dir = str(Path().resolve().parents[2] / "tests" / "data_octree")
(origin, vox_size, tiff_dims) = upload_skeleton.get_volume_info(data_dir, num_res)
vol = upload_skeleton.create_skeleton_layer(
    "file://"+dest_dir_skel, vox_size, tiff_dims, num_res
)
skeletons, segids = upload_skeleton.create_skel_segids(swc_dir, origin)
for skel in skeletons:
    vol.skeleton.upload(skel)

vol.skeleton.upload(skel)

Uploading: 100%|██████████| 1/1 [00:00<00:00, 143.76it/s]


In [17]:
ngl_sess = NeuroglancerSession(mip = 1, url = "file://" + dest_dir)

In [18]:
# ngl_sess.pull_voxel(2, 6, nx=10, ny=10, nz=10) # currently mip mismatch and scale mismatch
from cloudvolume import Bbox


In [19]:
def get_local_volume_around_vertex(ngl_sess, SEGID=2, VID=6, radius=10):
    skel = ngl_sess.cv.skeleton.get(SEGID)
    vertex = skel.vertices[VID]
    # below necessary to compensate for MIP differences
    # because the .swc file assumes 7 levels of resolution, 
    # but our test data only has the first 2
    scales = np.multiply(ngl_sess.cv.scales[1]["resolution"],2**3) # 3 for mip 1, 4 for mip 0
    voxel = np.round(np.divide(vertex, scales)).astype(int)
    bounds = Bbox(voxel, voxel)
    seed = bounds.to_list()
    shape = [radius, radius, radius]
    bounds = Bbox(np.subtract(seed[:3], shape), np.add(np.add(seed[3:], shape), 1))
    img = ngl_sess.cv.download(bounds)
    return img

In [20]:
# img = get_local_volume_around_vertex(ngl_sess, VID=50)
img = ngl_sess.pull_voxel(2, 50)
# 9891, 1102, 3449

Downloading: 100%|██████████| 1/1 [00:00<00:00, 319.06it/s]


OutOfBoundsError: Bbox([0, 0, 0],[1056, 800, 416], dtype=int32) did not fully contain the specified bounding box Bbox([4944, 550, 1724],[4947, 553, 1727], dtype=int32).

In [25]:
img2 = ngl_sess.pull_bounds_img(Bbox(
    (0,0,0), (int(9891/2**5), int(1102/2**5), int(3449/2**5))
    ))
img2 = ngl_sess.pull_bounds_img(Bbox(
    (0,0,0), (528, 400, 208)
))

Downloading:   0%|          | 0/32 [00:00<?, ?it/s]


In [26]:
with napari.gui_qt():
    ngl_sess.napari_viewer(img2)

remove TF  
allow stuff to run on Windows  
add more tests (actual unit tests!)

 - zoomed in retrieval of image
 - whole brain in napari

overlay SWC visualization (verify swc location, test!)  
(this is where it should be == this is where it is)

Why is the subvolume so bright?  
Janky with incomplete octree
 - swc file assumes 7 levels of resolution, we only have 2
 - interesting bug maybe? VID=100, radius=50
Use floats (possible cloudvolume PR)